<a href="https://colab.research.google.com/github/YongWanJin/KDTteam5_semiproject/blob/main/code/%EC%83%81%EA%B6%8C%EB%B3%80%ED%99%94%EC%A7%80%ED%91%9C_%ED%96%89%EC%A0%95%EB%8F%99%EB%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 파일 넣어둔 폴더로 이동
%cd /content/drive/My Drive

/content/drive/My Drive


In [22]:
import json
import pandas as pd

# 지도 시각화 참고 링크
# https://teddylee777.github.io/visualization/folium

# 서울 행정동 json raw파일(githubcontent) 불러오기
seoul_geo = json.load(open('seoul-dong.geojson', encoding='utf-8'))

In [ ]:
#서울 행정동별 boundary 시각화
import folium
m = folium.Map(location=[37.559819, 126.963895], zoom_start=11)
folium.GeoJson(seoul_geo, name='행정동').add_to(m)
m

In [3]:
import pandas as pd
biz_area_chg = pd.read_csv('/content/drive/My Drive/서울시 우리마을가게 상권분석서비스(행정동별 상권변화지표).csv'
                     , nrows=4664, encoding='cp949', dtype='object')
biz_area_chg.tail()

,기준_년_코드,기준_분기_코드,행정동_코드,행정동_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
4659,2019,3,11110560,평창동,HH,정체,113,59,109,53
4660,2019,3,11110550,부암동,LH,상권확장,100,58,109,53
4661,2019,3,11110540,삼청동,LH,상권확장,92,55,109,53
4662,2019,3,11110530,사직동,HH,정체,122,59,109,53
4663,2019,3,11110515,청운효자동,LH,상권확장,100,59,109,53


In [4]:
#필요한 컬럼만 추출
biz_area_chg = biz_area_chg[['기준_년_코드', '기준_분기_코드','행정동_코드', '행정동_코드_명','상권_변화_지표_명']]
biz_area_chg.tail()

,기준_년_코드,기준_분기_코드,행정동_코드,행정동_코드_명,상권_변화_지표_명
4659,2019,3,11110560,평창동,정체
4660,2019,3,11110550,부암동,상권확장
4661,2019,3,11110540,삼청동,상권확장
4662,2019,3,11110530,사직동,정체
4663,2019,3,11110515,청운효자동,상권확장


In [5]:
#분기별 분류를 위해 '기준 년'과 '기분 분기' 합침
biz_area_chg['기준_코드'] = biz_area_chg[['기준_년_코드', '기준_분기_코드']].apply('/Q'.join, axis=1)
biz_area_chg.tail()

,기준_년_코드,기준_분기_코드,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드
4659,2019,3,11110560,평창동,정체,2019/Q3
4660,2019,3,11110550,부암동,상권확장,2019/Q3
4661,2019,3,11110540,삼청동,상권확장,2019/Q3
4662,2019,3,11110530,사직동,정체,2019/Q3
4663,2019,3,11110515,청운효자동,상권확장,2019/Q3


In [6]:
#'기준 년'과 '기분 분기' 열 제거
biz_area_chg = biz_area_chg.drop(['기준_년_코드', '기준_분기_코드'], axis=1)
biz_area_chg.tail()

,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드
4659,11110560,평창동,정체,2019/Q3
4660,11110550,부암동,상권확장,2019/Q3
4661,11110540,삼청동,상권확장,2019/Q3
4662,11110530,사직동,정체,2019/Q3
4663,11110515,청운효자동,상권확장,2019/Q3


In [7]:
data = biz_area_chg.copy()

In [8]:
#<행정동별 상권변화지표> 중복 데이터 확인
data = biz_area_chg.groupby('행정동_코드_명').count()
data.sort_values(by='기준_코드', ascending=False)

,행정동_코드,상권_변화_지표_명,기준_코드
행정동_코드_명,,,
신사동,22,22,22
가락1동,11,11,11
옥수동,11,11,11
오류2동,11,11,11
오류1동,11,11,11
...,...,...,...
방학2동,11,11,11
방학1동,11,11,11
방이2동,11,11,11


In [14]:
#중복데이터가 존재하는 신사동 전체 row 확인 > merge 시 '행정동_코드_명'이 아닌 '행정동_코드'로 수행
sin = biz_area_chg[(biz_area_chg['행정동_코드_명'] == '신사동')]
sin.sort_values(by='행정동_코드', ascending=False)

,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드
66,11680510,신사동,다이나믹,2022/Q1
3034,11680510,신사동,다이나믹,2020/Q2
490,11680510,신사동,다이나믹,2021/Q4
4306,11680510,신사동,다이나믹,2019/Q3
914,11680510,신사동,다이나믹,2021/Q3
1338,11680510,신사동,다이나믹,2021/Q2
3882,11680510,신사동,다이나믹,2019/Q4
1762,11680510,신사동,다이나믹,2021/Q1
2186,11680510,신사동,다이나믹,2020/Q4
3458,11680510,신사동,다이나믹,2020/Q1


In [16]:
#행정동코드명의 겹치는 신사동 이름변경 필요 >> 실제 데이터는 수정되지 않는 문제 해결 필요
data = biz_area_chg.copy()
sin = data.loc[(data['행정동_코드_명']=='신사동')&(data['행정동_코드']=='11680510')]
sin.replace('신사동', '신사동(관악구)')

,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드
66,11680510,신사동(관악구),다이나믹,2022/Q1
490,11680510,신사동(관악구),다이나믹,2021/Q4
914,11680510,신사동(관악구),다이나믹,2021/Q3
1338,11680510,신사동(관악구),다이나믹,2021/Q2
1762,11680510,신사동(관악구),다이나믹,2021/Q1
2186,11680510,신사동(관악구),다이나믹,2020/Q4
2610,11680510,신사동(관악구),다이나믹,2020/Q3
3034,11680510,신사동(관악구),다이나믹,2020/Q2
3458,11680510,신사동(관악구),다이나믹,2020/Q1
3882,11680510,신사동(관악구),다이나믹,2019/Q4


In [17]:
data[(data['행정동_코드_명'] == '신사동')]
#sin.sort_values(by='행정동_코드', ascending=False)

,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드
66,11680510,신사동,다이나믹,2022/Q1
92,11620685,신사동,다이나믹,2022/Q1
490,11680510,신사동,다이나믹,2021/Q4
516,11620685,신사동,다이나믹,2021/Q4
914,11680510,신사동,다이나믹,2021/Q3
940,11620685,신사동,다이나믹,2021/Q3
1338,11680510,신사동,다이나믹,2021/Q2
1364,11620685,신사동,다이나믹,2021/Q2
1762,11680510,신사동,다이나믹,2021/Q1
1788,11620685,신사동,다이나믹,2021/Q1


In [27]:
biz_area_chg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4664 entries, 0 to 4663
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정동_코드      4664 non-null   object
 1   행정동_코드_명    4664 non-null   object
 2   상권_변화_지표_명  4664 non-null   object
 3   기준_코드       4664 non-null   object
dtypes: object(4)
memory usage: 145.9+ KB


In [19]:
# 행정동_코드 type 변경
biz_area_chg['행정동_코드'] = biz_area_chg['행정동_코드'].astype('int64')
biz_area_chg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4664 entries, 0 to 4663
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정동_코드      4664 non-null   int64 
 1   행정동_코드_명    4664 non-null   object
 2   상권_변화_지표_명  4664 non-null   object
 3   기준_코드       4664 non-null   object
dtypes: int64(1), object(3)
memory usage: 145.9+ KB


In [20]:
#자치구와 행정동 연결을 위한 파일 read
import pandas as pd
gu = pd.read_csv('서울생활이동데이터_서울자치구코드_20210907.csv', usecols=["시군구", "name"], encoding='cp949')
gu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구     25 non-null     int64 
 1   name    25 non-null     object
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


In [48]:
gu.head()

,시군구,name
0,11010,종로구
1,11020,중구
2,11030,용산구
3,11040,성동구
4,11050,광진구


In [21]:
dong = pd.read_csv('서울생활이동데이터_서울행정동코드_20210907.csv', usecols=["시군구", '읍면동',"name"], encoding='cp949')
dong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구     424 non-null    int64 
 1   읍면동     424 non-null    int64 
 2   name    424 non-null    object
dtypes: int64(2), object(1)
memory usage: 10.1+ KB


In [50]:
dong.head()

,시군구,읍면동,name
0,11010,1101053,사직동
1,11010,1101054,삼청동
2,11010,1101055,부암동
3,11010,1101056,평창동
4,11010,1101057,무악동


In [22]:
# gu & dong 합침
gu_dong = pd.merge(gu, dong, left_on='시군구', right_on='시군구')
gu_dong.head()

,시군구,name_x,읍면동,name_y
0,11010,종로구,1101053,사직동
1,11010,종로구,1101054,삼청동
2,11010,종로구,1101055,부암동
3,11010,종로구,1101056,평창동
4,11010,종로구,1101057,무악동


In [63]:
biz_area_chg.head()

,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드
0,11740700,둔촌2동,상권축소,2022/Q1
1,11740690,둔촌1동,정체,2022/Q1
2,11740685,길동,다이나믹,2022/Q1
3,11740660,성내3동,상권축소,2022/Q1
4,11740650,성내2동,다이나믹,2022/Q1


In [23]:
#상권변화지표와 시군구 데이터 합침
biz_loc = pd.merge(biz_area_chg, gu_dong, left_on='행정동_코드_명', right_on='name_y')
biz_loc.head()

,행정동_코드,행정동_코드_명,상권_변화_지표_명,기준_코드,시군구,name_x,읍면동,name_y
0,11740700,둔촌2동,상권축소,2022/Q1,11250,강동구,1125071,둔촌2동
1,11740700,둔촌2동,상권축소,2021/Q4,11250,강동구,1125071,둔촌2동
2,11740700,둔촌2동,상권축소,2021/Q3,11250,강동구,1125071,둔촌2동
3,11740700,둔촌2동,상권축소,2021/Q2,11250,강동구,1125071,둔촌2동
4,11740700,둔촌2동,상권축소,2021/Q1,11250,강동구,1125071,둔촌2동


In [24]:
#신사동 중복 데이터 확인  >> 각각의 파일명 변경 필요
test = biz_loc.copy()
test = test.groupby('행정동_코드_명').count()
test.sort_values(by='기준_코드', ascending=False)

,행정동_코드,상권_변화_지표_명,기준_코드,시군구,name_x,읍면동,name_y
행정동_코드_명,,,,,,,
신사동,44,44,44,44,44,44,44
가락1동,11,11,11,11,11,11,11
아현동,11,11,11,11,11,11,11
오류2동,11,11,11,11,11,11,11
오류1동,11,11,11,11,11,11,11
...,...,...,...,...,...,...,...
방학1동,11,11,11,11,11,11,11
방이2동,11,11,11,11,11,11,11
방이1동,11,11,11,11,11,11,11


In [65]:
biz_loc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4609 entries, 0 to 4608
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정동_코드      4609 non-null   int64 
 1   행정동_코드_명    4609 non-null   object
 2   상권_변화_지표_명  4609 non-null   object
 3   기준_코드       4609 non-null   object
 4   시군구         4609 non-null   int64 
 5   name_x      4609 non-null   object
 6   읍면동         4609 non-null   int64 
 7   name_y      4609 non-null   object
dtypes: int64(3), object(5)
memory usage: 324.1+ KB


In [ ]:
biz_pivot2 = biz_loc.pivot(index=['gu','dong'], columns='기준_코드', values='상권_변화_지표_명')
biz_pivot2

기준_코드     2019/Q3 2019/Q4 2020/Q1 2020/Q2 2020/Q3 2020/Q4 2021/Q1 2021/Q2  \
gu  dong                                                                    
강남구 개포1동       정체      정체      정체      정체      정체      정체      정체      정체   
    개포2동       정체      정체      정체      정체      정체      정체      정체      정체   
    개포4동     상권축소    상권축소    상권축소    상권축소    상권축소    상권축소    상권축소    상권축소   
    논현1동     다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹   
    논현2동     다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹   
...           ...     ...     ...     ...     ...     ...     ...     ...   
중랑구 상봉제2동    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹   
    신내1동     다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹   
    신내2동     다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹    다이나믹   
    중화제1동    다이나믹    상권축소    상권축소    다이나믹    다이나믹    상권축소    다이나믹    다이나믹   
    중화제2동    상권축소    상권축소    상권축소    상권축소    상권축소    상권축소    상권축소    상권축소   

기준_코드     2021/Q3 2021/Q4 2022/Q1  
gu  dong                           
강남구 개포1동       정체      정체      정체  
    개포2동       정체      정체      정체  
    개포4동     상권축소    상권축소    상권축소  
    논현1동     다이나믹    다이나믹    다이나믹  
    논현2동     다이나믹    다이나믹    다이나믹  
...           ...     ...     ...  
중랑구 상봉제2동    다이나믹    다이나믹    다이나믹  
    신내1동     다이나믹    다이나믹    다이나믹  
    신내2동     다이나믹    다이나믹    다이나믹  
    중화제1동    다이나믹    다이나믹    다이나믹  
    중화제2동    상권축소    상권축소    상권축소  

[418 rows x 11 columns]

In [ ]:
m = folium.Map(location=[37.559819, 126.963895], zoom_start=11)
folium.Choropleth(geo_data=seoul_geo, data=biz_pivot2, columns=('dong', '2019/Q3'), key_on = 'feature.properties.동',
                  fill_color='BuPu', legend_name='상권_변화_지표_명').add_to(m))

SyntaxError: ignored